# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] 1452076541 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /Users/zhusiqi/anaconda/envs/dato-env/lib/python2.7/site-packages/certifi/cacert.pem
1452076541 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to szhu21@jhu.edu and will expire on October 12, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-27792 - Server binary: /Users/zhusiqi/anaconda/envs/dato-env/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1452076541.log
[INFO] GraphLab Server Version: 1.7.1


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 21613
PROGRESS: Number of features          : 17
PROGRESS: Number of unpacked features : 17
PROGRESS: Number of coefficients    : 18
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000002  | 1.398632     | 6962915.603493     | 426631.749026 |
PROGRESS: | 2         | 3        | 0.000002  | 1.433980     | 6843144.200219     | 392488.929838 |
PROGRESS: | 3         | 4      

Find what features had non-zero weight.

In [6]:
print model_all.get('coefficients').print_rows(18)

+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None |  274873.05595 |
|     bedrooms     |  None |      0.0      |
| bedrooms_square  |  None |      0.0      |
|    bathrooms     |  None | 8468.53108691 |
|   sqft_living    |  None | 24.4207209824 |
| sqft_living_sqrt |  None | 350.060553386 |
|     sqft_lot     |  None |      0.0      |
|  sqft_lot_sqrt   |  None |      0.0      |
|      floors      |  None |      0.0      |
|  floors_square   |  None |      0.0      |
|    waterfront    |  None |      0.0      |
|       view       |  None |      0.0      |
|    condition     |  None |      0.0      |
|      grade       |  None | 842.068034898 |
|    sqft_above    |  None | 20.0247224171 |
|  sqft_basement   |  None |      0.0      |
|     yr_built     |  None |      0.0      |
|   yr_renovated   |  None |      0.0      |
+------------------+-------+---------------+
[18 rows x

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [7]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [8]:
import numpy as np
RSS = np.zeros((13,1))
i = 0
for l1_penalty in  np.logspace(1, 7, num=13):
    model_L1 = graphlab.linear_regression.create(training, target = 'price', features = all_features,
                                                 validation_set = None, 
                                                 l2_penalty = 0., l1_penalty = l1_penalty, verbose = False)
    residuals = model_L1.predict(validation) - validation['price']
    RSS[i] = (residuals**2).sum()
    i += 1
print RSS

[[  6.25766285e+14]
 [  6.25766285e+14]
 [  6.25766286e+14]
 [  6.25766288e+14]
 [  6.25766295e+14]
 [  6.25766317e+14]
 [  6.25766387e+14]
 [  6.25766607e+14]
 [  6.25767303e+14]
 [  6.25769508e+14]
 [  6.25776518e+14]
 [  6.25799063e+14]
 [  6.25883719e+14]]


In [9]:
print RSS.argmin()

0


In [10]:
model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty = 10.0, verbose = False)
print model.get("coefficients").print_rows(num_rows = 18)

+------------------+-------+------------------+
|       name       | index |      value       |
+------------------+-------+------------------+
|   (intercept)    |  None |  18993.4272128   |
|     bedrooms     |  None |  7936.96767903   |
| bedrooms_square  |  None |  936.993368193   |
|    bathrooms     |  None |  25409.5889341   |
|   sqft_living    |  None |  39.1151363797   |
| sqft_living_sqrt |  None |  1124.65021281   |
|     sqft_lot     |  None | 0.00348361822299 |
|  sqft_lot_sqrt   |  None |  148.258391011   |
|      floors      |  None |   21204.335467   |
|  floors_square   |  None |  12915.5243361   |
|    waterfront    |  None |  601905.594545   |
|       view       |  None |  93312.8573119   |
|    condition     |  None |  6609.03571245   |
|      grade       |  None |  6206.93999188   |
|    sqft_above    |  None |  43.2870534193   |
|  sqft_basement   |  None |  122.367827534   |
|     yr_built     |  None |  9.43363539372   |
|   yr_renovated   |  None |  56.0720034

*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [11]:
residuals_test = model_L1.predict(testing) - testing['price']
RSS_test = (residuals_test**2).sum()
print RSS_test

1.56972779669e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [12]:
model_L1_best = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty = 10, verbose = False)
print model_L1_best.get("coefficients").print_rows(num_rows = 18)

model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

+------------------+-------+------------------+
|       name       | index |      value       |
+------------------+-------+------------------+
|   (intercept)    |  None |  18993.4272128   |
|     bedrooms     |  None |  7936.96767903   |
| bedrooms_square  |  None |  936.993368193   |
|    bathrooms     |  None |  25409.5889341   |
|   sqft_living    |  None |  39.1151363797   |
| sqft_living_sqrt |  None |  1124.65021281   |
|     sqft_lot     |  None | 0.00348361822299 |
|  sqft_lot_sqrt   |  None |  148.258391011   |
|      floors      |  None |   21204.335467   |
|  floors_square   |  None |  12915.5243361   |
|    waterfront    |  None |  601905.594545   |
|       view       |  None |  93312.8573119   |
|    condition     |  None |  6609.03571245   |
|      grade       |  None |  6206.93999188   |
|    sqft_above    |  None |  43.2870534193   |
|  sqft_basement   |  None |  122.367827534   |
|     yr_built     |  None |  9.43363539372   |
|   yr_renovated   |  None |  56.0720034

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [13]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [14]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [15]:
import numpy as np
non_zero = np.zeros((20, 2))
i = 0
for l1_penalty in  l1_penalty_values:
    model_L12 = graphlab.linear_regression.create(training, target = 'price', features = all_features,
                                                  validation_set = None, 
                                                  l2_penalty = 0., l1_penalty = l1_penalty, verbose = False)
    non_zero[i, 0] = l1_penalty
    non_zero[i, 1] = model_L12['coefficients']['value'].nnz()  
    i += 1
print non_zero

[[  1.00000000e+08   1.80000000e+01]
 [  1.27427499e+08   1.80000000e+01]
 [  1.62377674e+08   1.80000000e+01]
 [  2.06913808e+08   1.80000000e+01]
 [  2.63665090e+08   1.70000000e+01]
 [  3.35981829e+08   1.70000000e+01]
 [  4.28133240e+08   1.70000000e+01]
 [  5.45559478e+08   1.70000000e+01]
 [  6.95192796e+08   1.70000000e+01]
 [  8.85866790e+08   1.60000000e+01]
 [  1.12883789e+09   1.50000000e+01]
 [  1.43844989e+09   1.50000000e+01]
 [  1.83298071e+09   1.30000000e+01]
 [  2.33572147e+09   1.20000000e+01]
 [  2.97635144e+09   1.00000000e+01]
 [  3.79269019e+09   6.00000000e+00]
 [  4.83293024e+09   5.00000000e+00]
 [  6.15848211e+09   3.00000000e+00]
 [  7.84759970e+09   1.00000000e+00]
 [  1.00000000e+10   1.00000000e+00]]


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [16]:
l1_penalty_min = non_zero[non_zero[:,1] >= 10][-1, 0]
l1_penalty_max = non_zero[non_zero[:,1] < 10][0, 0]
print l1_penalty_min
print l1_penalty_max

2976351441.63
3792690190.73


***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [17]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [18]:
RSS_2 = np.zeros((20, 3))
i = 0
for l1_penalty in l1_penalty_values:
    model_L13 = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    residuals = validation['price'] - model_L13.predict(validation)
    RSS_2[i, 0] = (residuals**2).sum()
    RSS_2[i, 1] = model_L13['coefficients']['value'].nnz()
    RSS_2[i, 2] = l1_penalty
    i += 1
print RSS_2

[[  9.66925692e+14   1.00000000e+01   2.97635144e+09]
 [  9.74019450e+14   1.00000000e+01   3.01931664e+09]
 [  9.81188368e+14   1.00000000e+01   3.06228184e+09]
 [  9.89328342e+14   1.00000000e+01   3.10524703e+09]
 [  9.98783211e+14   1.00000000e+01   3.14821223e+09]
 [  1.00847717e+15   1.00000000e+01   3.19117743e+09]
 [  1.01829878e+15   1.00000000e+01   3.23414263e+09]
 [  1.02824799e+15   1.00000000e+01   3.27710782e+09]
 [  1.03461691e+15   8.00000000e+00   3.32007302e+09]
 [  1.03855474e+15   8.00000000e+00   3.36303822e+09]
 [  1.04323724e+15   8.00000000e+00   3.40600341e+09]
 [  1.04693749e+15   7.00000000e+00   3.44896861e+09]
 [  1.05114763e+15   7.00000000e+00   3.49193381e+09]
 [  1.05599274e+15   7.00000000e+00   3.53489901e+09]
 [  1.06079953e+15   7.00000000e+00   3.57786420e+09]
 [  1.06570769e+15   6.00000000e+00   3.62082940e+09]
 [  1.06946434e+15   6.00000000e+00   3.66379460e+09]
 [  1.07350455e+15   6.00000000e+00   3.70675980e+09]
 [  1.07763278e+15   6.00000

In [19]:
print RSS_2[RSS_2[:, 1] == max_nonzeros, :][0]

[  1.04693749e+15   7.00000000e+00   3.44896861e+09]


***QUIZ QUESTIONS***

1. What value of l1_penalty in our narrow range has the lowest RSS on the VALIDATION set and has sparsity equal to max_nonzeros?
2. What features in this model have non-zero coefficients?

In [20]:
L1 = RSS_2[RSS_2[:, 1] == max_nonzeros, :][0, 2]

In [21]:
model = graphlab.linear_regression.create(training, target = 'price', features = all_features,
                                              validation_set = None, verbose = False,
                                              l2_penalty = 0., l1_penalty = L1)
non_zero_2 = model["coefficients"][model["coefficients"]["value"]]
non_zero_2.print_rows()

+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None | 222253.192544 |
|     bedrooms     |  None | 661.722717782 |
|    bathrooms     |  None | 15873.9572593 |
|   sqft_living    |  None | 32.4102214513 |
| sqft_living_sqrt |  None | 690.114773313 |
|      grade       |  None | 2899.42026975 |
|    sqft_above    |  None | 30.0115753022 |
+------------------+-------+---------------+
[7 rows x 3 columns]

